In [36]:
%matplotlib inline
from importlib import reload
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [37]:
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

## Setup

We haven't really looked into the detail of how this works yet - so this is provided for self-study for those who are interested. We'll look at it closely next week.

In [38]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [39]:
!tail {path} -n25

are thinkers who believe in the saints.


144

It stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. There are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. Others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of God and hence felt himself sinless; so that
through his imagination--that should not be too harshly judged since the
whole of antiquity swarmed with sons of god--he attained the same goal,
the sense of complete sinlessness, complete irresponsibility, that can
now be attained by every individual through science.--In the same manner
I have viewed the saints of India

In [40]:
#path = 'data/wiki/'
#text = open(path+'small.txt').read().lower()
#print('corpus length:', len(text))

#text = text[0:1000000]

In [41]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 58


In [42]:
chars.insert(0, "\0")

In [43]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwx'

In [44]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [45]:
idx = [char_indices[c] for c in text]

In [46]:
idx[:10]

[43, 45, 32, 33, 28, 30, 32, 1, 1, 1]

In [47]:
''.join(indices_char[i] for i in idx[:70])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

## Preprocess and create model

In [48]:
maxlen = 40
sentences = []
next_chars = []
for i in range(0, len(idx) - maxlen+1):
    sentences.append(idx[i: i + maxlen])
    next_chars.append(idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))

nb sequences: 600854


In [49]:
np.array(sentences).shape

(600854, 40)

In [50]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [51]:
sentences.shape, next_chars.shape

((600852, 40), (600852, 40))

In [52]:
n_fac = 24

In [53]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen),
        LSTM(512,return_sequences=True, dropout=0.2, recurrent_dropout=0.2,
             implementation=2),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2,
             implementation=2),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])    

In [54]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

## Train

In [55]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [56]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=640 * 5, epochs=1)

Epoch 1/1
600852/600852 [==============================] - 437s - loss: 2.9903   


In [57]:
print_example()

ethics is a basic foundation of all that gotl mhes nfse as or arabrel ds ind af h ege teen whep giv os-af oc ve kem ah inilehy "ite
gperees lar. an0 coserluthebcara. ond,
sepste,,
op gnpum nrun nhs tx eugor ed o tiriregior she' taiwh the ans ho tisese) o
ser
c ghads tig
nhrmr=,, ticit shaseris anc, thlr the the bod th,  eo monsesep miciood
aadled "eacges abe


In [58]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=640 * 5, epochs=1)

Epoch 1/1
600852/600852 [==============================] - 437s - loss: 2.2895   


In [59]:
print_example()

ethics is a basic foundation of all that and somanity thates. i anver isticiolm whould apeny cerelieg, sor deld be kinticals as e cad
undorser, a cucs of ghism i
mede not
aly srugity
sird
in gofity thee ale wlot the distrencars it evonged to ta is,icy a7
ite ollessting mor natufe increts, os va the sond not vath sik frocs: mesere whuelt, an beyter ourder--up


In [60]:
model.optimizer.lr=0.001

In [63]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=640 * 5, epochs=1)

Epoch 1/1
600852/600852 [==============================] - 436s - loss: 1.6796   


In [237]:
print_example()

ethics is a basic foundation of all that schools pedhaps a new prisons of the ashamed in which
a coverbine estimates of the assumption that one avoid; he will curse about pain:
     people, he-equally present to
the lalier,
nature. that he has
rendered and henceforth distrain and impulses to perceive that each other
former and dangerous, and cannot at
the pu


In [250]:
model.optimizer.lr=0.0001

In [239]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 639s - loss: 1.2892   


In [240]:
print_example()

ethics is a basic foundation of all that account has its granitify them.

131. the new "dilence," out of the
same light,
interpretation thereof: under the "thinking"
there, to counter-arguments in the monality, so many language:
though
all nobilitys of higher impulses, man and hence to everything of seldom man.



chapter i. woman decides according the injur


In [242]:
model.save_weights('data/char_rnn.h5')

In [257]:
model.optimizer.lr=0.00001

In [243]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 640s - loss: 1.2544   


In [22]:
print_example()

ethics is a basic foundation of all that it is because without which not not une
sense
science, and powerful to the errous mankind of other, to becilualizing
of kand of curtes and to imt wouth the great recogning precisely, as upon wishes a soos of viod
of a person and clussion, owing jan fin our good of different
seffering with onese-bying does no danger an


In [258]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 640s - loss: 1.2218   


In [264]:
print_example()

ethics is a basic foundation of all that
the belief in the importance. the employs concerning
seriousness and
materialism, it is circles which alone is already attained, that he sees
also the day after thinking
of mankind, brightness, resistance--and after the value of "nature" in order to nevertheless
have taken a system of liberal fatalists are willing him


In [283]:
print_example()

ethics is a basic foundation of all that were beought by the temptation of truth--for the rest of a sublime medely and take part of life, which lacks himself the
credibility about this, in short, and raise such a
gods; and on the
other hand, the explanation of
the case, as the most ingredient, and insight, and approach as to the
peculiarly prolonged "distrus


In [282]:
model.save_weights('data/char_rnn.h5')